In [1]:
from awips.dataaccess import DataAccessLayer

# Set host
DataAccessLayer.changeEDEXHost("edex-cloud.unidata.ucar.edu")

# Init data request
request = DataAccessLayer.newDataRequest()

# Set datatype
request.setDatatype("grid")

# getAvailableLocationNames method will return a list of all available models
available_grids = DataAccessLayer.getAvailableLocationNames(request)
for grid in available_grids:
    print grid
    

Populating the interactive namespace from numpy and matplotlib
GFSLAMP5
GFSGuide
ECMF6
ECMF3
ECMF4
HPCqpfNDFD
ECMF-Global
RAP13
ECMF5
RTMA5
AVN211
NamDNG
ECMF9
QPE-MSR
estofsUS
ECMF12
ECMF10
ECMF8
MRMS_1000
ETA
LAMP2p5
HRRR
URMA25
GFS40
GFS
RAP40
RTMA
AVN225
NAM12
ECMF1
ECMF2
ECMF11
DGEX
NamDNG5
estofsPR
NCWF
ECMF7
NOHRSC-SNOW
HPCGuide
NAM40


LocationNames is different for different plugins
- radar is icao
- satellite is sector


In [16]:
import numpy as np
from awips.dataaccess import DataAccessLayer

DataAccessLayer.changeEDEXHost("edex-cloud.unidata.ucar.edu")
request = DataAccessLayer.newDataRequest()
request.setDatatype("grid")

# Grid request
request.setLocationNames('RAP40')
request.setParameters("T")
request.setLevels("500MB")

# Get available times
t = DataAccessLayer.getAvailableTimes(request)

# Select last available time [-1]
response = DataAccessLayer.getGridData(request, [t[-1]])
data = response[0]

# Print info
print data.getLocationName()
print t[-1]
print 'Unit is', data.getUnit()
print 'Parameter is', data.getParameter()

# Print data array
print data.getRawData()

# Print lat/lon array
lon,lat = data.getLatLonCoords()
print "lat array =", lat
print "lon array =", lon

RAP40
2016-02-23 15:00:00 (12)
Unit is K
Parameter is T
[[ 273.89916992  273.64916992  272.89916992 ...,  250.77416992
   250.77416992  250.64916992]
 [ 250.52416992  250.52416992  250.77416992 ...,  258.89916992
   258.64916992  258.39916992]
 [ 258.27416992  257.89916992  257.39916992 ...,  269.77416992
   269.64916992  269.52416992]
 ..., 
 [ 290.89916992  290.02416992  289.89916992 ...,  294.64916992
   294.14916992  293.77416992]
 [ 294.39916992  295.02416992  295.77416992 ...,  293.39916992
   293.14916992  293.02416992]
 [ 292.77416992  292.64916992  292.52416992 ...,  289.02416992
   289.39916992  289.39916992]]
lat array = [[ 54.24940109  54.35071945  54.45080566 ...,  57.9545517   57.91926193
   57.88272858]
 [ 57.84495163  57.80593109  57.76566696 ...,  58.07667542  58.08861542
   58.09931183]
 [ 58.10876846  58.11697769  58.12394714 ...,  56.40270996  56.46187973
   56.51980972]
 ..., 
 [ 19.93209648  19.89832115  19.86351395 ...,  20.054636    20.06362152
   20.07156372]
 

# Plotting a Grid

With **matplotlib** and **basemap**, we can plot an a re-projected grid image:


In [ ]:
%matplotlib inline
import sys, cStringIO, os, os.path, datetime
import random
import string
import matplotlib.tri as mtri
import matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy
from mpl_toolkits.basemap import Basemap, cm
import numpy as np
from numpy import linspace, transpose
from numpy import meshgrid
from awips.dataaccess import DataAccessLayer

#from pint import UnitRegistry

# If we don't specify an EDEX host, default is "edex-cloud.unidata.ucar.edu"
DataAccessLayer.changeEDEXHost("edex-cloud.unidata.ucar.edu")
request = DataAccessLayer.newDataRequest()
request.setDatatype("grid")

gridTimeIndex = -1

# EDEX Data Access Framework
request = DataAccessLayer.newDataRequest()
request.setDatatype("maps")
request.setParameters("state")
request.addIdentifier("locationField","name")
request.addIdentifier("geomField","the_geom")
request.addIdentifier("table","mapdata.states")
request.setLocationNames("Colorado")
response = DataAccessLayer.getGeometryData(request, None)
# Now set area
request = DataAccessLayer.newDataRequest()
request.setEnvelope(response[0].getGeometry().buffer(3))
# Now query grid
request.setDatatype("grid")
request.setLocationNames('RAP40')
request.setParameters("T")
request.setLevels("500MB")
t = DataAccessLayer.getAvailableTimes(request)

response = DataAccessLayer.getGridData(request, [t[gridTimeIndex]])
data = response[0]
lons,lats = data.getLatLonCoords()

map = Basemap(projection='cyl',
      resolution = 'c',
      llcrnrlon = lons.min(), llcrnrlat = lats.min(),
      urcrnrlon =lons.max(), urcrnrlat = lats.max()
)
map.drawcoastlines()
map.drawstates()
map.drawcountries()
x = linspace(0, map.urcrnrx, data.getRawData().shape[1])
y = linspace(0, map.urcrnry, data.getRawData().shape[0])
xx, yy = meshgrid(x, y)

ngrid = len(x)
rlons = np.repeat(np.linspace(np.min(lons), np.max(lons), ngrid),
          ngrid).reshape(ngrid, ngrid)
rlats = np.repeat(np.linspace(np.min(lats), np.max(lats), ngrid),
          ngrid).reshape(ngrid, ngrid).T
tli = mtri.LinearTriInterpolator(mtri.Triangulation(lons.flatten(),
          lats.flatten()), data.getRawData().flatten())
rdata = tli(rlons, rlats)
clevs = [40,45,50,55,60,65,70,75,80,85,90,95]
cs = map.pcolormesh(rlons, rlats, rdata-272.15, latlon=True, vmin=-10, vmax=-2)

# add colorbar.
cbar = map.colorbar(cs,location='bottom',pad="5%")
cbar.set_label(data.getParameter() + "(F)")
plt.show()
